## Import

In [1]:
import os
os.system('pip install tasknet pygrove xpflow setGPU')

0

In [2]:
import pandas as pd
import numpy as np
import tasknet as tn
from tqdm.notebook import tqdm
from dataclasses import dataclass
tqdm.pandas()
import random
from datasets import DatasetDict
import datasets
from torch.utils.data import Dataset
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer, AutoTokenizer
import inspect
import torch
import transformers
from types import MethodType
os.environ["WANDB_DISABLED"] = "true"
os.environ['CUDA_MODULE_LOADING'] = 'LAZY'
import tensorflow as tf
from collections import Counter
import requests
import re
import funcy as fc

In [3]:
tf.config.list_physical_devices('GPU')

[]

In [4]:
import importlib
importlib.reload(tn)

<module 'tasknet' from 'c:\\Users\\Quent\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\tasknet\\__init__.py'>

# Loading Data

## WVMCQ7

In [5]:
df_wvs = pd.read_json("https://raw.githubusercontent.com/QuentinTilman/PollBERT-paper/master/Datasets/WVMCQ7/WVMCQ7-Normalized_labels_percentage_wise.json")

## RedditPoll350K

In [6]:
stop_words = ["able","about","above","abroad","according","accordingly","across","actually","adj","after","afterwards","again","against","ago","ahead","ain't","all","allow","allows","almost","alone","along","alongside","already","also","although","always","am","amid","amidst","among","amongst","an","and","another","any","anybody","anyhow","anyone","anything","anyway","anyways","anywhere","apart","appear","appreciate","appropriate","are","aren't","around","as","a's","aside","ask","asking","associated","at","available","away","awfully","back","backward","backwards","be","became","because","become","becomes","becoming","been","before","beforehand","begin","behind","being","believe","below","beside","besides","best","better","between","beyond","both","brief","but","by","came","can","cannot","cant","can't","caption","cause","causes","certain","certainly","changes","clearly","c'mon","co","co.","com","come","comes","concerning","consequently","consider","considering","contain","containing","contains","corresponding","could","couldn't","course","c's","currently","dare","daren't","definitely","described","despite","did","didn't","different","directly","do","does","doesn't","doing","done","don't","down","downwards","during","each","edu","eg","eight","eighty","either","else","elsewhere","end","ending","enough","entirely","especially","et","etc","even","ever","evermore","every","everybody","everyone","everything","everywhere","ex","exactly","example","except","fairly","far","farther","few","fewer","fifth","first","five","followed","following","follows","for","forever","former","formerly","forth","forward","found","four","from","further","furthermore","get","gets","getting","given","gives","go","goes","going","gone","got","gotten","greetings","had","hadn't","half","happens","hardly","has","hasn't","have","haven't","having","he","he'd","he'll","hello","help","hence","her","here","hereafter","hereby","herein","here's","hereupon","hers","herself","he's","hi","him","himself","his","hither","hopefully","how","howbeit","however","hundred","i'd","ie","if","ignored","i'll","i'm","immediate","in","inasmuch","inc","inc.","indeed","indicate","indicated","indicates","inner","inside","insofar","instead","into","inward","is","isn't","it","it'd","it'll","its","it's","itself","i've","just","k","keep","keeps","kept","know","known","knows","last","lately","later","latter","latterly","least","less","lest","let","let's","like","liked","likely","likewise","little","look","looking","looks","low","lower","ltd","made","mainly","make","makes","many","may","maybe","mayn't","me","mean","meantime","meanwhile","merely","might","mightn't","mine","minus","miss","more","moreover","most","mostly","mr","mrs","much","must","mustn't","my","myself","name","namely","nd","near","nearly","necessary","need","needn't","needs","neither","never","neverf","neverless","nevertheless","new","next","nine","ninety","no","nobody","non","none","nonetheless","noone","no-one","nor","normally","not","nothing","notwithstanding","novel","now","nowhere","obviously","of","off","often","oh","ok","okay","old","on","once","one","ones","one's","only","onto","opposite","or","other","others","otherwise","ought","oughtn't","our","ours","ourselves","out","outside","over","overall","own","particular","particularly","past","per","perhaps","placed","please","plus","possible","presumably","probably","provided","provides","que","quite","qv","rather","rd","re","really","reasonably","recent","recently","regarding","regardless","regards","relatively","respectively","right","round","said","same","saw","say","saying","says","second","secondly","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sensible","sent","serious","seriously","seven","several","shall","shan't","she","she'd","she'll","she's","should","shouldn't","since","six","so","some","somebody","someday","somehow","someone","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specified","specify","specifying","still","sub","such","sup","sure","take","taken","taking","tell","tends","th","than","thank","thanks","thanx","that","that'll","thats","that's","that've","the","their","theirs","them","themselves","then","thence","there","thereafter","thereby","there'd","therefore","therein","there'll","there're","theres","there's","thereupon","there've","these","they","they'd","they'll","they're","they've","thing","things","think","third","thirty","this","thorough","thoroughly","those","though","three","through","throughout","thru","thus","till","to","together","too","took","toward","towards","tried","tries","truly","try","trying","t's","twice","two","un","under","underneath","undoing","unfortunately","unless","unlike","unlikely","until","unto","up","upon","upwards","us","use","used","useful","uses","using","usually","v","value","various","versus","very","via","viz","vs","want","wants","was","wasn't","way","we","we'd","welcome","well","we'll","went","were","we're","weren't","we've","what","whatever","what'll","what's","what've","when","whence","whenever","where","whereafter","whereas","whereby","wherein","where's","whereupon","wherever","whether","which","whichever","while","whilst","whither","who","who'd","whoever","whole","who'll","whom","whomever","who's","whose","why","will","willing","wish","with","within","without","wonder","won't","would","wouldn't","yes","yet","you","you'd","you'll","your","you're","yours","yourself","yourselves","you've","zero","a","how's","i","when's","why's","b","c","d","e","f","g","h","j","l","m","n","o","p","q","r","s","t","u","uucp","w","x","y","z","I","www","amount","bill","bottom","call","computer","con","couldnt","cry","de","describe","detail","due","eleven","empty","fifteen","fifty","fill","find","fire","forty","front","full","give","hasnt","herse","himse","interest","itse”","mill","move","myse”","part","put","show","side","sincere","sixty","system","ten","thick","thin","top","twelve","twenty","abst","accordance","act","added","adopted","affected","affecting","affects","ah","announce","anymore","apparently","approximately","aren","arent","arise","auth","beginning","beginnings","begins","biol","briefly","ca","date","ed","effect","et-al","ff","fix","gave","giving","heres","hes","hid","home","id","im","immediately","importance","important","index","information","invention","itd","keys","kg","km","largely","lets","line","'ll","means","mg","million","ml","mug","na","nay","necessarily","nos","noted","obtain","obtained","omitted","ord","owing","page","pages","poorly","possibly","potentially","pp","predominantly","present","previously","primarily","promptly","proud","quickly","ran","readily","ref","refs","related","research","resulted","resulting","results","run","sec","section","shed","shes","showed","shown","showns","shows","significant","significantly","similar","similarly","slightly","somethan","specifically","state","states","stop","strongly","substantially","successfully","sufficiently","suggest","thered","thereof","therere","thereto","theyd","theyre","thou","thoughh","thousand","throug","til","tip","ts","ups","usefully","usefulness","'ve","vol","vols","wed","whats","wheres","whim","whod","whos","widely","words","world","youd","youre"]

In [7]:
r = requests.get("https://raw.githubusercontent.com/dariusk/corpora/master/data/religion/religions.json")
religions = r.json()
beliefs = []
for major in [item for item in religions.values()]:
    majors = [m for m in major]
    beliefs.extend(majors)
    try:
        for minor in major.values():
            for religion in minor:
                beliefs.append(religion)
    except:
        next
beliefs.extend(["atheist","christian","religious","non-religious","not religious"])

In [8]:
r = requests.get("https://raw.githubusercontent.com/Imagin-io/country-nationality-list/master/countries.json")
dictionary = r.json()
df_dictionary = pd.DataFrame(dictionary)
def nation(item):
    item["en_short_name"] = " "+item["en_short_name"].lower()+" "
    item["alpha_2_code"] = " "+item["alpha_2_code"].lower()+" "
    item["alpha_3_code"] = " "+item["alpha_3_code"].lower()+" "
    nationality = re.split(', | or',item["nationality"].lower())
    item["nationality"] = nationality
    return item
df_dictionary=df_dictionary.progress_apply(nation,axis=1)

countries = []
countries.extend(df_dictionary["en_short_name"].values)
countries.extend(df_dictionary["alpha_2_code"].values)
countries.extend(df_dictionary["alpha_3_code"].values)

  0%|          | 0/249 [00:00<?, ?it/s]

In [9]:
nationalities = []
for item in df_dictionary["nationality"].values:
    for label in item:
        nationalities.append(label)

locations = []
locations.extend(df_dictionary["en_short_name"].values)
locations.extend(df_dictionary["alpha_2_code"].values)
locations.extend(df_dictionary["alpha_3_code"].values)
locations.extend([item.lower() for item in ["Asia", "Africa", "North America", "South America", "Antarctica", "Europe", "Australia","rest of the world","usa","us","eu","america",'other country']])

In [10]:
df_reddit= pd.read_json('https://raw.githubusercontent.com/QuentinTilman/PollBERT-paper/master/Datasets/RedditPoll350K/RedditPoll350K_categorized.json')
del df_reddit['Unnamed: 0']

# Data Preprocessing

## Preprocessing Methods

In [11]:
@dataclass
class DataCollatorForPolls:
    tokenizer: None=None
    tokenizer_kwargs: None=None
    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)]
            for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        pad_args=inspect.signature(self.tokenizer.pad).parameters.keys()
        batch = self.tokenizer.pad(flattened_features, **fc.project(self.tokenizer_kwargs,pad_args))
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.stack(labels).float() #### Allows one value per choice
        return batch

In [12]:
def apply_question_template(df):
  #we need to take into account that the input to the model is Demographic + Question + Answer
  # we thus need to create a question template that seems natural
  # This template is only necessary if the input is not already in a question format
  demographic = df["demographic"]
  category = df["demographic_category"]
  subject = df["input"]
  result = any(item in subject for item in ["?","How","Why","Where","When","Who","What"])
  sentence = f" {demographic}: {subject} ?".lower()
  if(not result):
    if(category == "age"):
        sentence = f" {demographic} years old: which of these options would you take in regard to the subject '{subject} ?'".lower()  
    else:
        sentence = f" {demographic}: which of these options would you take in regard to the subject '{subject} ?'".lower()  
  df['question'] = sentence
  return df

def apply_question_template_without_demo(df):
  #we need to take into account that the input to the model is Demographic + Question + Answer
  # we thus need to create a question template that seems natural
  # This template is only necessary if the input is not already in a question format
  subject = df["input"]
  result = any(item in subject for item in ["?","How","Why","Where","When","Who","What"])
  sentence = f" {subject} ?".lower()
  if(not result):
    sentence = f"which of these options would you take in regard to the subject '{subject} ?'".lower()  
  df['question'] = sentence
  return df

### WVMCQ7

### RedditPoll350K

In [13]:
def get_demographics(x):
    parenthesized=[s[s.find("(")+1:s.rfind(")")]  for s in x.options.keys()]
    parenthesized=[s for s in parenthesized if s]
    parenthesized_count=pd.Series.value_counts(parenthesized)
    return list(parenthesized_count[parenthesized_count>1].index)

In [14]:
def extract_demographic(x):
    demographics = get_demographics(x)
    x['demographic']=''
    if not demographics:
        return x
    if len(demographics)>1:
        print('failure:',demographics, x.options)
        return x
    demographic=demographics[0]
    x['options'] = {k.replace(f'({demographic})',''):v for (k,v) in x.options.items()}
    x['demographic']=demographic
    return x

In [15]:
def split_questions(x):
    demographics=get_demographics(x)

    l=[]
    for d in demographics:
        l+=[fc.project(x.options, [x for x in x.options if f'({d})' in x])] #if row has multiple demographics its repeated in dataset?
    if l:
        x.options=l
        x['split']=True
    else:
        x.options=[x.options]
        x['split']=False
    return x


In [16]:
num_choices=6
choices_names=[f'choice_{i}' for i in range(num_choices)]
mode='proba'

def preprocess(x):
    choices = (list(x.options.keys())+['']*num_choices)[:num_choices]
    for i,n in enumerate(choices_names):
        x[n]=choices[i]
    scores = np.array(list(x.options.values())+[-1]*num_choices)[:num_choices]*1.0
    if mode=='binary':
        x['label'] = np.random.choice(np.flatnonzero(scores == scores.max()))
    else:
        scores[scores!=-1]= scores[scores!=-1]/scores[scores!=-1].sum()
        x['label'] = scores
    del x['options']
    x['task']=0
    return x

In [17]:
def filter_nb_answers(data):
  dataset = []
  keys = list(data[["input","demographic","year"]].value_counts().keys())
  for (input,demographic,year) in keys:
      results = data.groupby(["input","demographic","year"]).get_group((input,demographic,year))
      if( len(results["label"]) > 1 and len(results["label"]) <= 6):
        options = {}
        for i in range(len(results["choice"])):
          options[list(results["choice"])[i]] = list(results["label"])[i]

        dataset.append({"input":input,"demographic":demographic,"year":year,"options":options})
  return pd.DataFrame(dataset) 

In [18]:
def choices(x):
    choices = (x["options"]+['']*num_choices)[:num_choices]
    scores = np.array(list(x.labels)+[-1]*num_choices)[:num_choices]*1.0
    for i,n in enumerate(choices_names):
        x[n]=choices[i]  
    x["labels"] = scores
    return x

In [19]:
def filter_data(x):
    options = x["options"]
    if(1 < len(options) <7):
        return x

In [20]:
def get_split(x):
    rnd=random.Random(x).random()
    if rnd<0.95:
        return 'train'
    if rnd<0.975:
        return 'validation'
    else:
        return 'test'

In [21]:
def normalize_nationality(x):
    terms = ["man","guy","and"] + stop_words
    label = x["demographic"].lower()
    if (2 <= len(label) <= 3) and label not in terms:
      if any(token.replace(" ","").lower() in label.split(" ") for token in nationalities):
        token = [token for token  in nationalities if token.replace(" ","").lower() in label.split(" ")]
        if("not" in label) | ("non" in label):
              label = "not "+token[0]
        else:
              label = token[0]
    elif any(token.lower() in label.split(" ") for token in nationalities):
        token = [token for token  in nationalities if token.lower() in label.split(" ")]
        if("not" in label) | ("non" in label):
            label = "not "+token[0]
        else:
            label = token[0]
    elif any(token.lower() in label.split("-") for token in nationalities):
        token = [token for token  in nationalities if token.lower() in label.split("-")]
        if("not" in label) | ("non" in label):
            label = "not "+token[0]
        else:
            label = token[0]
            
    x["demographic"] = label
    return x

In [22]:
def normalize_countries(x):
    terms = ["man","guy","and"] + stop_words
    label = x["demographic"].lower()
    if (2 <= len(label) <= 3) and label not in terms:
      if any(token.replace(" ","").lower() in label.split(" ") for token in countries):
        token = [token for token  in countries if token.replace(" ","").lower() in label.split(" ")]
        if("not" in label) | ("non" in label):
              label = "not "+token[0]
        else:
              label = token[0]
    elif any(token.lower() in label.split(" ") for token in countries):
        token = [token for token  in countries if token.lower() in label.split(" ")]
        if("not" in label) | ("non" in label):
            label = "not "+token[0]
        else:
            label = token[0]
    elif any(token.lower() in label.split("-") for token in countries):
        token = [token for token  in countries if token.lower() in label.split("-")]
        if("not" in label) | ("non" in label):
            label = "not "+token[0]
        else:
            label = token[0]
    x["demographic"] = label
    return x

In [23]:
def normalize_genders(x):
    label = x["demographic"].lower()
    female= ["f","female","woman","chick","girl"]
    male = ["m","male","man","guy","men"]
    if any(token.lower() in female for token in label.split(" ")) | any(token.lower() in female for token in label.split("-")):
        if ("not" in label) | ("non" in label):
            label = "not female"
        else:
            label = "female"
    if any(token in male for token in label.split(" ")) | any(token in male for token in label.split("-")):
        if("not" in label) | ("non" in label):
            label = "not male"
        else:
            label = "male"
    x["demographic"] = label
    return x

In [24]:
def normalize_politics(x):
    label = x["demographic"].lower()
    right = ["d","democratic","right",'right-leaning']
    left = ["republican","republicans","left","left-leaning","r"]
    if any(token.lower() in label.split(" ") for token in right) | any(token.lower() in label.split("-") for token in right):
        if("not" in label) | ("non" in label):
            label = "not democratic"
        else: 
            label = "democratic"
    elif any(token.lower() in label.split(" ") for token in left) | any(token.lower() in label.split("-") for token in left):
        if("not" in label) | ("non" in label):
            label = "not republican"
        else: 
            label = "republican"
    
    x["demographic"] = label
    return x

In [25]:
def to_percentage(x):
    options = x["options"]
    try:
        x["nb_votes"] = sum(options.values())
        x["options"] = {key:(value/sum(options.values())) for key,value in options.items()}
        return x
    except:
        pass

In [26]:
def get_nationality(x):
    demo = x["demographic"]
    category = x["demographic_category"]
    if any(token in demo for token in nationalities):
        category = "nationality"
    x["demographic_category"] = category
    return x

In [27]:
def get_countries(x):
    demo = x["demographic"]
    category = x["demographic_category"]
    if any(token in demo for token in locations):
        category = "location"
    x["demographic_category"] = category
    return x

In [28]:
def get_genders(x):
    demo = x["demographic"]
    category = x["demographic_category"]
    female= ["f","female","woman","chick","girl","women"]
    male = ["m","male","man","guy","boy","men"]
    alphabetplus = [item.lower() for item in ["straight","non-binary","nonbinary","non binary","queer","trans","transgender","Pansexual","asexual","skoliosexual","cisgender","genderqueer","intersex","gay","homosexual","lesbian"]]
    if any(token in female for token in demo.split()):
        category = "gender"
    if any(token in male for token in demo.split()):
        category = "gender"
    if any(token in alphabetplus for token in demo.split()):
        category = "gender"
    x["demographic_category"] = category
    return x

In [29]:
def get_politics(x):
    demo = x["demographic"]
    category = x["demographic_category"]

    right = ["d","democratic","right","r",'right-leaning',"liberal","democrat","right-wing"]
    left = ["republican","republicans","left","left-leaning","conservative","left-wing"]
    if any(token in demo.split(" ") for token in right):
        category = "politics"
    if any(token in demo.split(" ") for token in left):
        category = "politics"
    x["demographic_category"] = category
    return x

In [30]:
def get_diet(x):
    demo = x["demographic"]
    category = x["demographic_category"]
    diets = ["vegan","vegetarian","meat-eater"]
    if any(token in demo.split(" ") for token in diets):
        category = "diet"
    x["demographic_category"] = category
    return x

In [31]:
def has_digit(x):
    demo = x["demographic"]
    category = x["demographic_category"]
    if re.search(r'\d+', demo):
            category = "digit "+demo
    x["demographic_category"] = category
    return x

In [32]:
def get_years(x):
    demo = x["demographic"]
    category = x["demographic_category"]
    if re.search(r'\d+', demo):
            if 2023 > int(re.search(r'\d+', demo).group()) > 1000:
                category = "years"
    x["demographic_category"] = category
    return x

In [33]:
def get_age(x):
    demo = x["demographic"]
    category = x["demographic_category"]
    if (str(category) != "nan"):
        if("digit" in category):
            if any(item in demo for item in ["y/o",">","<","+","below","under","over","years","yo","y.o","younger","older","born",'student','teenager',"adult",'gen z','gen y','gen x']):
                category = "age"
    x["demographic_category"] = category
    return x

In [34]:
def get_ethnicity(x):
    demo = x["demographic"]
    category = x["demographic_category"]
    colours = ["white","black","caucasian","dusky","orange-colored","yellow","red","indian","malay","mongoloid","carib","nergoid"]
    if any(colour in demo for colour in colours):
        category = "ethnicity"
    x['demographic_category'] = category
    return x

In [35]:
def get_belief(x):
    demo = x["demographic"]
    category = x["demographic_category"]
    if any(b in demo for b in beliefs):
        category = "belief"
    x["demographic_category"] = category
    return x

In [36]:
def get_other(x):
    category = x["demographic_category"]
    if str(category) == "nan" or "digit" in category:
        category = "other"
    x["demographic_category"] = category
    return x

In [37]:
def remove_nan(x):
  labels = x["label"]
  if any(np.isnan(labels)):
      pass
  else:
      return x

## RedditPoll350K

In [38]:
df_reddit_split=df_reddit.progress_apply(split_questions,axis=1).explode('options')

  0%|          | 0/312306 [00:00<?, ?it/s]

In [39]:
df_reddit_split=df_reddit_split.progress_apply(extract_demographic,axis=1)

  0%|          | 0/322543 [00:00<?, ?it/s]

failure: ['165 cm', "165 cm) - 6' (183 cm"] {'< 5\' 5" (165 cm), Male': 14, '< 5\' 5" (165 cm), Female': 34, '5\' 5" (165 cm) - 6\' (183 cm), Male': 106, '5\' 5" (165 cm) - 6\' (183 cm), Female': 36}
failure: ["165 cm) - 6' (183 cm", '183 cm'] {'5\' 5" (165 cm) - 6\' (183 cm), Male': 106, '5\' 5" (165 cm) - 6\' (183 cm), Female': 36, "> 6' (183 cm), Male": 65, "> 6' (183 cm), Female": 4}


In [40]:
df_reddit_demo = df_reddit_split[df_reddit_split.demographic.map(len).map(bool)]

In [41]:
df_reddit_demographics_normalized_labels=df_reddit_demo.progress_apply(normalize_nationality,axis=1)
df_reddit_demographics_normalized_labels=df_reddit_demographics_normalized_labels.progress_apply(normalize_genders,axis=1)
df_reddit_demographics_normalized_labels=df_reddit_demographics_normalized_labels.progress_apply(normalize_countries,axis=1)
df_reddit_demographics_normalized_labels=df_reddit_demographics_normalized_labels.progress_apply(normalize_politics,axis=1)

  0%|          | 0/20214 [00:00<?, ?it/s]

  0%|          | 0/20214 [00:00<?, ?it/s]

  0%|          | 0/20214 [00:00<?, ?it/s]

  0%|          | 0/20214 [00:00<?, ?it/s]

In [42]:
df_reddit_percentage = df_reddit_demographics_normalized_labels.progress_apply(preprocess,axis=1)
df_reddit_percentage = df_reddit_percentage.dropna()

  0%|          | 0/20214 [00:00<?, ?it/s]

C:\Users\Quent\AppData\Local\Temp\ipykernel_34288\2768567453.py:13: RuntimeWarning: invalid value encountered in divide
  scores[scores!=-1]= scores[scores!=-1]/scores[scores!=-1].sum()
C:\Users\Quent\AppData\Local\Temp\ipykernel_34288\2768567453.py:13: RuntimeWarning: invalid value encountered in divide
  scores[scores!=-1]= scores[scores!=-1]/scores[scores!=-1].sum()
C:\Users\Quent\AppData\Local\Temp\ipykernel_34288\2768567453.py:13: RuntimeWarning: invalid value encountered in divide
  scores[scores!=-1]= scores[scores!=-1]/scores[scores!=-1].sum()
C:\Users\Quent\AppData\Local\Temp\ipykernel_34288\2768567453.py:13: RuntimeWarning: invalid value encountered in divide
  scores[scores!=-1]= scores[scores!=-1]/scores[scores!=-1].sum()
C:\Users\Quent\AppData\Local\Temp\ipykernel_34288\2768567453.py:13: RuntimeWarning: invalid value encountered in divide
  scores[scores!=-1]= scores[scores!=-1]/scores[scores!=-1].sum()
C:\Users\Quent\AppData\Local\Temp\ipykernel_34288\2768567453.py:13: Ru

In [43]:
df_reddit_percentage["demographic_category"] = np.NaN
df_reddit_category = df_reddit_percentage.progress_apply(has_digit,axis=1)
df_reddit_category = df_reddit_category.progress_apply(get_age,axis=1)
df_reddit_category = df_reddit_category.progress_apply(get_years,axis=1)
df_reddit_category=df_reddit_category.progress_apply(get_nationality,axis=1)
df_reddit_category=df_reddit_category.progress_apply(get_genders,axis=1)
df_reddit_category=df_reddit_category.progress_apply(get_countries,axis=1)
df_reddit_category=df_reddit_category.progress_apply(get_politics,axis=1)
df_reddit_category=df_reddit_category.progress_apply(get_diet,axis=1)
df_reddit_category=df_reddit_category.progress_apply(get_ethnicity,axis=1)
df_reddit_category=df_reddit_category.progress_apply(get_belief,axis=1)
df_reddit_category=df_reddit_category.progress_apply(get_other,axis=1)

  0%|          | 0/20214 [00:00<?, ?it/s]

  0%|          | 0/20214 [00:00<?, ?it/s]

  0%|          | 0/20214 [00:00<?, ?it/s]

  0%|          | 0/20214 [00:00<?, ?it/s]

  0%|          | 0/20214 [00:00<?, ?it/s]

...

  0%|          | 0/20214 [00:00<?, ?it/s]

  0%|          | 0/20214 [00:00<?, ?it/s]

  0%|          | 0/20214 [00:00<?, ?it/s]

  0%|          | 0/20214 [00:00<?, ?it/s]

  0%|          | 0/20214 [00:00<?, ?it/s]

In [44]:
counter = [x for x in Counter(df_reddit_category["demographic"]).keys() if Counter(df_reddit_category["demographic"])[x] >= 10]
df_reddit_demographics = df_reddit_category[df_reddit_category["demographic"].isin(counter)]

In [45]:
if mode=='binary':
    df_reddit_demographics_normalized=df_reddit_demographics[df_reddit_demographics.label<num_choices]
else:
    df_reddit_demographics_normalized=df_reddit_demographics[df_reddit_demographics.label.map(len)==num_choices]
len(df_reddit_demographics_normalized)

13460

In [46]:
df_reddit_demographics_normalized.dropna

<bound method DataFrame.dropna of             id                                              title     created  \
1       yvae7l  Do You Believe Your Congressional District is ...  1668455219   
1       yvae7l  Do You Believe Your Congressional District is ...  1668455219   
65      yuipdn                            Does body count matter?  1668382874   
65      yuipdn                            Does body count matter?  1668382874   
66      yui5re                      are your nails long or short?  1668381574   
...        ...                                                ...         ...   
308670  gkzer2                               Lying about your age  1589653023   
309958  gdk0w1                          Your preference in comics  1588625918   
311046  g3cqyd         Would you rather have Fascism or Communism  1587164986   
311046  g3cqyd         Would you rather have Fascism or Communism  1587164986   
311887  fsruvs                                 political ideology  15857121

In [47]:
df_reddit_demographics_normalized[df_reddit_demographics_normalized.label.progress_map(lambda x: any(np.isnan(a) for a in x))]
df_reddit_demographics_normalized['split']=df_reddit_demographics_normalized.title.map(get_split)

  0%|          | 0/13460 [00:00<?, ?it/s]

In [48]:
df_reddit_demographics_normalized

,id,title,created,category,split,demographic,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,label,task,demographic_category
1,yvae7l,Do You Believe Your Congressional District is ...,1668455219,news_&_social_concern,train,democratic,Yes,No,,,,,"[0.6730769230769231, 0.3269230769230769, -1.0,...",0,politics
1,yvae7l,Do You Believe Your Congressional District is ...,1668455219,news_&_social_concern,train,republican,Yes,No,,,,,"[0.4666666666666667, 0.5333333333333333, -1.0,...",0,politics
65,yuipdn,Does body count matter?,1668382874,diaries_&_daily_life,train,male,Yes,No,,,,,"[0.5737704918032787, 0.4262295081967213, -1.0,...",0,gender
65,yuipdn,Does body count matter?,1668382874,diaries_&_daily_life,train,female,Yes,No,,,,,"[0.2826086956521739, 0.717391304347826, -1.0, ...",0,gender
66,yui5re,are your nails long or short?,1668381574,fashion_&_style,test,male,Long,Short,,,,,"[0.06680584551148225, 0.9331941544885177, -1.0...",0,gender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308670,gkzer2,Lying about your age,1589653023,diaries_&_daily_life,train,male,Yes,No,,,,,"[0.42276422764227645, 0.5772357723577236, -1.0...",0,gender
309958,gdk0w1,Your preference in comics,1588625918,arts_&_culture,train,comment,Multiple,Other,,,,,"[0.44, 0.56, -1.0, -1.0, -1.0, -1.0]",0,other
311046,g3cqyd,Would you rather have Fascism or Communism,1587164986,news_&_social_concern,validation,republican,Fascism,Communism,,,,,"[0.15714285714285714, 0.8428571428571429, -1.0...",0,politics
311046,g3cqyd,Would you rather have Fascism or Communism,1587164986,news_&_social_concern,validation,democratic,Fascism,Communism,,,,,"[0.631578947368421, 0.3684210526315789, -1.0, ...",0,politics


In [49]:
df_reddit_demographics_normalized_labels = df_reddit_demographics_normalized.progress_apply(remove_nan,axis=1)
df_reddit_demographics_normalized_labels.dropna()

  0%|          | 0/13460 [00:00<?, ?it/s]

,id,title,created,category,split,demographic,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,label,task,demographic_category
1,yvae7l,Do You Believe Your Congressional District is ...,1.668455e+09,news_&_social_concern,train,democratic,Yes,No,,,,,"[0.6730769230769231, 0.3269230769230769, -1.0,...",0.0,politics
1,yvae7l,Do You Believe Your Congressional District is ...,1.668455e+09,news_&_social_concern,train,republican,Yes,No,,,,,"[0.4666666666666667, 0.5333333333333333, -1.0,...",0.0,politics
65,yuipdn,Does body count matter?,1.668383e+09,diaries_&_daily_life,train,male,Yes,No,,,,,"[0.5737704918032787, 0.4262295081967213, -1.0,...",0.0,gender
65,yuipdn,Does body count matter?,1.668383e+09,diaries_&_daily_life,train,female,Yes,No,,,,,"[0.2826086956521739, 0.717391304347826, -1.0, ...",0.0,gender
66,yui5re,are your nails long or short?,1.668382e+09,fashion_&_style,test,male,Long,Short,,,,,"[0.06680584551148225, 0.9331941544885177, -1.0...",0.0,gender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308670,gkzer2,Lying about your age,1.589653e+09,diaries_&_daily_life,train,male,Yes,No,,,,,"[0.42276422764227645, 0.5772357723577236, -1.0...",0.0,gender
309958,gdk0w1,Your preference in comics,1.588626e+09,arts_&_culture,train,comment,Multiple,Other,,,,,"[0.44, 0.56, -1.0, -1.0, -1.0, -1.0]",0.0,other
311046,g3cqyd,Would you rather have Fascism or Communism,1.587165e+09,news_&_social_concern,validation,republican,Fascism,Communism,,,,,"[0.15714285714285714, 0.8428571428571429, -1.0...",0.0,politics
311046,g3cqyd,Would you rather have Fascism or Communism,1.587165e+09,news_&_social_concern,validation,democratic,Fascism,Communism,,,,,"[0.631578947368421, 0.3684210526315789, -1.0, ...",0.0,politics


In [224]:
df_show = df_reddit_demographics_normalized_labels.sample(frac=1).reset_index(drop=True)


4    How do you view the British Empire?
Name: title, dtype: object

In [228]:
df_show[4:5].to_json("RedditPoll350k_data_example.json")

### Reco

In [50]:
df_reddit_demographics_normalized_labels_demo = df_reddit_demographics_normalized_labels.copy()
df_reddit_demographics_normalized_labels_no_demo = df_reddit_demographics_normalized_labels.copy()

In [51]:
try:
    del df_reddit_demographics_normalized_labels['options']
except:
    pass
    
df_reddit_demographics_normalized_labels_demo['labels']=df_reddit_demographics_normalized_labels_demo['label']
del df_reddit_demographics_normalized_labels_demo['label']

df_reddit_demographics_normalized_labels_no_demo['labels']=df_reddit_demographics_normalized_labels_no_demo['label']
del df_reddit_demographics_normalized_labels_no_demo['label']


df_reddit_demographics_normalized_labels_demo['inputs']=df_reddit_demographics_normalized_labels.demographic+': '+df_reddit_demographics_normalized_labels.title
dataset_reddit_demographics_plbrt=DatasetDict({k:datasets.Dataset.from_pandas(df_reddit_demographics_normalized_labels_demo[df_reddit_demographics_normalized_labels_demo['split']==k]) for k in {'train','test','validation'}})


df_reddit_demographics_normalized_labels_no_demo['inputs']=df_reddit_demographics_normalized_labels.title
dataset_reddit__plbrt=DatasetDict({k:datasets.Dataset.from_pandas(df_reddit_demographics_normalized_labels_no_demo[df_reddit_demographics_normalized_labels_no_demo['split']==k]) for k in {'train','test','validation'}})

In [52]:
reco = tn.MultipleChoice(
    dataset_reddit__plbrt,
    s1="inputs",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
    data_collator=DataCollatorForPolls()
)

tasks_reddit__plbrt = [reco]

In [53]:
tasks_reddit__plbrt[0].dataset["validation"]["inputs"]

['When’s the last time you saw a medical doctor?',
 'Have you finished a book this year?',
 'Have you finished a book this year?',
 'Should the requirement to drink and gamble be based on responsibility and knowledge instead of age?',
 'Should the requirement to drink and gamble be based on responsibility and knowledge instead of age?',
 "Do you wan't a general war?",
 "Do you wan't a general war?",
 'Would an affair who another person cheats with be more likely to also cheat in their own relationship, if they had one in the future?',
 'Would an affair who another person cheats with be more likely to also cheat in their own relationship, if they had one in the future?',
 'Which one tastes better?',
 'Do you like being naked?',
 'Do you like being naked?',
 'If being Naked was more common in public spaces, would humans in general, work out more?',
 'If being Naked was more common in public spaces, would humans in general, work out more?',
 'Who should pay for dinner? (Pick the best out 

In [54]:
reco = tn.MultipleChoice(
    dataset_reddit_demographics_plbrt,
    s1="inputs",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
    data_collator=DataCollatorForPolls()
)

tasks_reddit_demographics_plbrt = [reco]

In [55]:
tasks_reddit_demographics_plbrt[0].dataset["validation"]["inputs"]

[' usa : When’s the last time you saw a medical doctor?',
 'male: Have you finished a book this year?',
 'female: Have you finished a book this year?',
 'american: Should the requirement to drink and gamble be based on responsibility and knowledge instead of age?',
 'not american: Should the requirement to drink and gamble be based on responsibility and knowledge instead of age?',
 "europe: Do you wan't a general war?",
 "america: Do you wan't a general war?",
 'male: Would an affair who another person cheats with be more likely to also cheat in their own relationship, if they had one in the future?',
 'female: Would an affair who another person cheats with be more likely to also cheat in their own relationship, if they had one in the future?',
 'any: Which one tastes better?',
 'male: Do you like being naked?',
 'female: Do you like being naked?',
 'male: If being Naked was more common in public spaces, would humans in general, work out more?',
 'female: If being Naked was more common

## WVS

In [56]:
df_wvs = df_wvs.progress_apply(filter_data,axis=1)
df_wvs = df_wvs.dropna()
pd.DataFrame(df_wvs)
df_wvs= pd.DataFrame([item for item in df_wvs.values if item[1].lower() not in ["-5","-4","-3","-2","-1","no answer","don't know"] ],columns= ["year","demographic","input","demographic_category","options","labels"])

  0%|          | 0/209542 [00:00<?, ?it/s]

In [57]:
df_wvs['split']=df_wvs.input.progress_map(get_split)

  0%|          | 0/179181 [00:00<?, ?it/s]

In [58]:
df_wvs = pd.DataFrame(df_wvs)
df_wvs

,year,demographic,input,demographic_category,options,labels,split
0,2017.0,18,Aims of country: first choice,age,"[A high level of economic growth, Making sure ...","[0.4291417166, 0.1397205589, 0.3233532934, 0.1...",train
1,2017.0,18,Aims of country: second choice,age,"[A high level of economic growth, Making sure ...","[0.2611336032, 0.2064777328, 0.3198380567, 0.2...",train
2,2017.0,18,Aims of respondent: first choice,age,"[maintaining order in the nation, Giving peopl...","[0.3605577689, 0.24103585660000001, 0.14541832...",train
3,2017.0,18,Aims of respondent: second choice,age,"[maintaining order in the nation, Giving peopl...","[0.2851405622, 0.2489959839, 0.218875502000000...",train
4,2017.0,18,Believe in: God,age,"[No, Yes]","[0.3611691023, 0.6388308977]",train
...,...,...,...,...,...,...,...
179176,2022.0,male,Vote in elections: local level,gender,"[Always, Usually, Never, Not allowed to vote]","[0.5963541667000001, 0.24804687500000003, 0.14...",test
179177,2022.0,male,Which party would you vote for: first choice (...,gender,"[No right to vote, Other, Independent Candidate]","[0.36250000000000004, 0.125, 0.5125000000000001]",train
179178,2022.0,male,Willingness to fight for country,gender,"[no, yes]","[0.22697899840000002, 0.7730210016]",train
179179,2022.0,male,Work is a duty towards society,gender,"[Strongly agree, Agree, Neither agree nor disa...","[0.3264913406, 0.3688261706, 0.1610006414, 0.1...",train


### Add choice columns to dataset

In [59]:
df_wvs=df_wvs.progress_apply(choices,axis=1)

  0%|          | 0/179181 [00:00<?, ?it/s]

In [60]:
if mode=='binary':
    df_wvs=df_wvs[df_wvs.labels<num_choices]
else:
    df_wvs=df_wvs[df_wvs.labels.map(len)==num_choices]
len(df_wvs)

179181

### Apply template for uniform questions


In [61]:
df_demo = df_wvs.progress_apply(apply_question_template,axis=1)
df_no_demo =  df_wvs.progress_apply(apply_question_template_without_demo,axis=1)

  0%|          | 0/179181 [00:00<?, ?it/s]

  0%|          | 0/179181 [00:00<?, ?it/s]

### Training, Validation and Test Sets

In [62]:
df_demo[df_demo.labels.progress_map(lambda x: any(np.isnan(a) for a in x))]
df_no_demo[df_no_demo.labels.progress_map(lambda x: any(np.isnan(a) for a in x))]

  0%|          | 0/179181 [00:00<?, ?it/s]

  0%|          | 0/179181 [00:00<?, ?it/s]

,year,demographic,input,demographic_category,options,labels,split,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,question


In [63]:
def get_split(x):
    rnd=random.Random(x).random()
    if rnd<0.95:
        return 'train'
    if rnd<0.975:
        return 'validation'
    else:
        return 'test'
    
df_demo['split']=df_demo.input.map(get_split)
try:
    del df_demo['options']
except:
    pass
dataset=DatasetDict({k:datasets.Dataset.from_pandas(df_demo[df_demo['split']==k]) for k in {'train','test','validation'}})


df_no_demo['split']=df_demo['split']
try:
    del df_no_demo['options']
except:
    pass
dataset=DatasetDict({k:datasets.Dataset.from_pandas(df_no_demo[df_no_demo['split']==k]) for k in {'train','test','validation'}})

In [64]:
reco = tn.MultipleChoice(
    dataset,
    s1="question",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
    data_collator=DataCollatorForPolls()
)

tasks_wvs_demo = [reco]

In [65]:
reco = tn.MultipleChoice(
    dataset,
    s1="question",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
    data_collator=DataCollatorForPolls()
)

tasks_wvs_no_demo = [reco]

# Load Model

In [66]:
from transformers.models.roberta.modeling_roberta import *
  
        
def forward(
    self,
    input_ids: Optional[torch.LongTensor] = None,
    token_type_ids: Optional[torch.LongTensor] = None,
    attention_mask: Optional[torch.FloatTensor] = None,
    labels: Optional[torch.LongTensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    head_mask: Optional[torch.FloatTensor] = None,
    inputs_embeds: Optional[torch.FloatTensor] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
) -> Union[Tuple[torch.Tensor], MultipleChoiceModelOutput]:
    r"""
    labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
        Labels for computing the multiple choice classification loss. Indices should be in `[0, ...,
        num_choices-1]` where `num_choices` is the size of the second dimension of the input tensors. (See
        `input_ids` above)
    """
    return_dict = return_dict if return_dict is not None else self.config.use_return_dict
    num_choices = input_ids.shape[1] if input_ids is not None else inputs_embeds.shape[1]

    flat_input_ids = input_ids.view(-1, input_ids.size(-1)) if input_ids is not None else None
    flat_position_ids = position_ids.view(-1, position_ids.size(-1)) if position_ids is not None else None
    flat_token_type_ids = token_type_ids.view(-1, token_type_ids.size(-1)) if token_type_ids is not None else None
    flat_attention_mask = attention_mask.view(-1, attention_mask.size(-1)) if attention_mask is not None else None
    flat_inputs_embeds = (
        inputs_embeds.view(-1, inputs_embeds.size(-2), inputs_embeds.size(-1))
        if inputs_embeds is not None
        else None
    )

    outputs = self.roberta(
        flat_input_ids,
        position_ids=flat_position_ids,
        token_type_ids=flat_token_type_ids,
        attention_mask=flat_attention_mask,
        head_mask=head_mask,
        inputs_embeds=flat_inputs_embeds,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
    )
    pooled_output = outputs[1]

    pooled_output = self.dropout(pooled_output)

    logits = self.classifier(pooled_output)

    reshaped_logits = logits.view(-1, num_choices)


    m=nn.Softmax(dim=1)
    reshaped_logits=m(reshaped_logits)
    loss = None
    if labels is not None:
        loss_fct = self.loss
        loss = loss_fct(reshaped_logits, labels)
    if not return_dict:
        output = (reshaped_logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output
    
    return MultipleChoiceModelOutput(
        loss=loss,
        logits=reshaped_logits,
        hidden_states=outputs.hidden_states,
        attentions=outputs.attentions,
    )
from types import MethodType 

In [67]:
from statistics import mean
def compute_metrics_old(element):
    f_distance = lambda x,y: np.abs((x-y)).max(axis=0).mean()
    y_pred,y_true = element
    distance = f_distance(y_pred,y_true)
    return distance
    
def crop(element):
  y_pred,y_true = element
  mask = y_true == -1
  return (y_pred[~mask],y_true[~mask])


def compute_metrics(self,batch):
  meta = {"name": self.name, "size": len(batch.predictions), "index": self.index}
  distance = mean([compute_metrics_old(crop(element)) for element in zip(batch.predictions,batch.label_ids)])
  return {**meta, 'distance':distance}


def mse_loss(input, target, ignored_index=-1):
    mask = target == ignored_index
    out = (input[~mask]-target[~mask])**2
    return out.mean()**.5

def bce_loss(input, target, ignored_index=-1):
  loss = nn.BCELoss()
  mask = target == ignored_index
  out = loss(input[~mask],target[~mask])
  return out

In [68]:
from transformers import AutoTokenizer, AutoModelForMultipleChoice

tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)

model = AutoModelForMultipleChoice.from_pretrained("Qubix/RoBERTa-PollBERT350K_full_model")

# Analysis

## Global Demographic Effect

In this part we study the effect of the demographic attribute by checking whether we can sse variations on the predictions of MCQ without Attribute and with Attribute

## Local Demographic Effect

In this part we study the effect of the demographic attribute by checking whether there is a noticeable difference in the prediction between different demographic attributes.

### Reddit

#### Load list Question with Significant Divisiveness

In [104]:
list_reddit = pd.read_json("https://raw.githubusercontent.com/QuentinTilman/PollBERT-paper/master/Datasets/RedditPoll350K/RedditPoll350K_significant_divisiveness_questions.json")
list_reddit

,demographic_A,demographic_B,questions
0,male,female,"[What do you do after peeing?, Should FDS be r..."
1,male,democratic,[Do you want to have kids?]
2,male,religious,[Do you believe in ghosts?]
3,male,women,"[Have you ever been sexually harassed?, What a..."
4,male,under 18,[Do you drink alcohol?]
5,male,non-binary,"[Have you ever been to therapy before?, how do..."
6,female,republican,[Do you want to have kids?]
7,female,democratic,[Do you want to have kids?]
8,female,women,[Do you shave your armpits?]
9,female,under 18,[Do you drink alcohol?]


#### Retrieving Common Polls

In [69]:
def test(x,dictionary_demo_question):
    demographics = list(x["demographic"])
    for i in range(len(demographics)):
        dictionary_demo_question[demographics[i]].append(list(x["title"])[i])
    return x

In [70]:
df_reddit_demographics_normalized_labels.drop(df_reddit_demographics_normalized_labels[df_reddit_demographics_normalized_labels["demographic_category"] == "other"].index,inplace=True)

In [79]:
counts = df_reddit_demographics_normalized_labels["demographic"].value_counts()
keys = [k for k in df_reddit_demographics_normalized_labels["demographic"].value_counts().keys() if counts[k] > 50]
df_reddit_category_reduced = pd.DataFrame([item for item in df_reddit_demographics_normalized_labels.values if item[5] in keys],columns = df_reddit_demographics_normalized_labels.columns)

In [80]:
dictionary_demo_question = {}
for key in df_reddit_category_reduced[df_reddit_category_reduced["split"] == "train"]["demographic"].value_counts().keys():
    dictionary_demo_question[key] = []

In [81]:
df_reddit_category_reduced_choices = df_reddit_category_reduced[df_reddit_category_reduced["split"] == "train"].groupby(["title"]).apply(test,dictionary_demo_question)

C:\Users\Quent\AppData\Local\Temp\ipykernel_34288\3658719877.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_reddit_category_reduced_choices = df_reddit_category_reduced[df_reddit_category_reduced["split"] == "train"].groupby(["title"]).apply(test,dictionary_demo_question)


In [208]:
df_reddit_category_reduced_choices

,id,title,created,category,split,demographic,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,label,task,demographic_category
0,yvae7l,Do You Believe Your Congressional District is ...,1.668455e+09,news_&_social_concern,train,democratic,Yes,No,,,,,"[0.6730769230769231, 0.3269230769230769, -1.0,...",0.0,politics
1,yvae7l,Do You Believe Your Congressional District is ...,1.668455e+09,news_&_social_concern,train,republican,Yes,No,,,,,"[0.4666666666666667, 0.5333333333333333, -1.0,...",0.0,politics
2,yuipdn,Does body count matter?,1.668383e+09,diaries_&_daily_life,train,male,Yes,No,,,,,"[0.5737704918032787, 0.4262295081967213, -1.0,...",0.0,gender
3,yuipdn,Does body count matter?,1.668383e+09,diaries_&_daily_life,train,female,Yes,No,,,,,"[0.2826086956521739, 0.717391304347826, -1.0, ...",0.0,gender
6,ytu61j,Do you think you’re lazy at least sometimes?,1.668322e+09,diaries_&_daily_life,train,male,Yes,No,,,,,"[0.9444444444444444, 0.05555555555555555, -1.0...",0.0,gender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9724,gtyqij,Are Americans too loving of their flag?,1.590930e+09,news_&_social_concern,train,not american,Yes,No,,,,,"[0.8130841121495327, 0.18691588785046728, -1.0...",0.0,location
9725,gtyqij,Are Americans too loving of their flag?,1.590930e+09,news_&_social_concern,train,american,Yes,No,,,,,"[0.5598958333333334, 0.4401041666666667, -1.0,...",0.0,location
9726,gn9fuo,Who do you think has it easier in life?,1.589973e+09,diaries_&_daily_life,train,male,Guys,Girls,,,,,"[0.55, 0.45, -1.0, -1.0, -1.0, -1.0]",0.0,gender
9727,gn9fuo,Who do you think has it easier in life?,1.589973e+09,diaries_&_daily_life,train,female,Guys,Girls,,,,,"[0.8214285714285714, 0.17857142857142858, -1.0...",0.0,gender


In [83]:
demo_pair_questions = {}
for k in dictionary_demo_question.keys():
    for l in dictionary_demo_question.keys():
        if(k != l):
            s = set(dictionary_demo_question[k]).intersection(set(dictionary_demo_question[l]))
            if(len(s) != 0):
                    if((l,k) not in demo_pair_questions.keys()):
                        demo_pair_questions[(k,l)] = set(dictionary_demo_question[k]).intersection(set(dictionary_demo_question[l]))

In [207]:
demo_pair_questions

{('male',
  'female'): {'If someone calls themselves a “nice guy” do you automatically assume they’re not a nice person to be around?', 'How long is the hair on your head?', 'Can you bench press 135 lbs/60 kg?', 'Should guys shave their balls?', "Do you think women's self defense actually works?", 'If a male birth control (not condoms or a vasectomy) became available, would you be supportive of it? And men, would you personally take it?', 'Would you help if you saw a woman being attacked by a person bigger than you?', 'is gossip a good thing?', 'What do you prefer, Doritos or Pringles?', 'Do you think a beard or any facial hair makes a man more handsome?', 'Do you usually wash your genitals after peeing?', 'Have you ever masturbated at a friends house during a sleepover or just hanging out?', 'Do you have anorexia?', 'Should men shave their armpit, chest and stomach hair?', 'Who make the best comedians?', 'What gender do you play as in red dead redemption 2 online?', 'Would you like to

In [85]:
df_reddit_category_reduced_choices[df_reddit_category_reduced_choices["title"]== 'Do you wear glasses?']

,id,title,created,category,split,demographic,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,label,task,demographic_category
5594,sesyta,Do you wear glasses?,1.643383e+09,diaries_&_daily_life,train,female,Yes,No,,,,,"[0.631578947368421, 0.3684210526315789, -1.0, ...",0.0,gender
7465,ok0213,Do you wear glasses?,1.626251e+09,diaries_&_daily_life,train,male,Yes,No,,,,,"[0.45542168674698796, 0.5445783132530121, -1.0...",0.0,gender
7466,ok0213,Do you wear glasses?,1.626251e+09,diaries_&_daily_life,train,female,Yes,No,,,,,"[0.6377551020408163, 0.3622448979591837, -1.0,...",0.0,gender


In [86]:
keys_question = df_reddit_category_reduced_choices[["title"]].value_counts().keys()

In [87]:
def predict(x):
    input = f'{x["demographic"]} : {x["title"]}'
    labels = [l for l in x["label"] if l != -1]
    choices = []
    question = []
    for i in range(len(labels)):
        index = f'choice_'+str(i)
        choices.append(x[index])
        question.append(input)
    encoding = tokenizer(question, choices, return_tensors="pt", padding=True)
    outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()}, labels=torch.tensor(labels).unsqueeze(0))
    return x

In [88]:
question = 'males do you wear glasses?'
choice1 = "yes"
choice2 = "no"
choice3 = ""
encoding = tokenizer([question,question], [choice1,choice2], return_tensors="pt", padding=True)
outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()})
outputs.logits.softmax(dim=-1).tolist()

[[0.4044351875782013, 0.5955648422241211]]

In [89]:
df_test_predict = df_reddit_demographics_normalized_labels[df_reddit_demographics_normalized_labels["split"] == "train"].progress_apply(predict,axis=1)

  0%|          | 0/10397 [00:00<?, ?it/s]

#### Using List

In [170]:
def to_sample(x):
    l = []
    demographic_A = x["demographic_A"]
    demographic_B = x["demographic_B"]
    questions = x['questions']
    for q in questions:
        l.append({"demographic_A":demographic_A,"demographic_B":demographic_B,"question":q})
    return l

In [177]:
l = []
for item in list_reddit.values:
    demographic_A = item[0]
    demographic_B = item[1]
    questions = item[2]
    for q in questions:
        l.append({"demographic_A":demographic_A,"demographic_B":demographic_B,"question":q})

In [180]:
sample_reddit = pd.DataFrame(l).sample(100)

In [186]:
sample_reddit

,demographic_A,demographic_B,question
24,male,female,Do men have an issue with dating women who don...
1919,american,not american,Would gun control benefit America? (As in the ...
1776,american,not american,Should citizen be allowed to have/carry firearm?
1713,american,not american,Do you think the American Dream exists? (Or ex...
1342,male,female,Do you think it’s okay to date your friend’s ex ?
...,...,...,...
1118,male,female,How much do you care about your partners body ...
1965,american,not american,Do you use counter-clockwise or anti-clockwise?
776,male,female,Do you chew your nails?
154,male,female,Would you rather be a


In [203]:
list(df_reddit_split[df_reddit_split["title"] == "Should citizen be allowed to have/carry firearm?"]["options"].values[0].keys())

{'Yes ': 220, 'No ': 72}

In [204]:
def predict(x):
    demoA = x["demographic_A"]
    demoB = x["demographic_B"]
    question = x["question"]
    options = list(df_reddit_split[df_reddit_split["title"] == "Should citizen be allowed to have/carry firearm?"]["options"].values[0].keys())
    

    qA = demoA+": "+question
    encoding = tokenizer([qA]*len(options), options, return_tensors="pt", padding=True)
    outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()})
    pred_A = outputs.logits.softmax(dim=-1).tolist()

    qB = demoB+": "+question
    encoding = tokenizer([qB]*len(options), options, return_tensors="pt", padding=True)
    outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()})
    pred_B = outputs.logits.softmax(dim=-1).tolist()

    x["pred_A"] = pred_A
    x["pred_B"] = pred_B
    return x

In [205]:
results_sample_reddit = sample_reddit.progress_apply(predict,axis = 1)

  0%|          | 0/100 [00:00<?, ?it/s]

In [206]:
results_sample_reddit

,demographic_A,demographic_B,question,pred_A,pred_B
24,male,female,Do men have an issue with dating women who don...,"[[0.46310096979141235, 0.5368990302085876]]","[[0.4632395803928375, 0.5367604494094849]]"
1919,american,not american,Would gun control benefit America? (As in the ...,"[[0.45022979378700256, 0.549770176410675]]","[[0.44382214546203613, 0.5561778545379639]]"
1776,american,not american,Should citizen be allowed to have/carry firearm?,"[[0.44641491770744324, 0.5535851120948792]]","[[0.4432397782802582, 0.5567602515220642]]"
1713,american,not american,Do you think the American Dream exists? (Or ex...,"[[0.4576976001262665, 0.5423023104667664]]","[[0.4689546823501587, 0.5310453772544861]]"
1342,male,female,Do you think it’s okay to date your friend’s ex ?,"[[0.4643760621547699, 0.5356238484382629]]","[[0.4646535813808441, 0.5353464484214783]]"
...,...,...,...,...,...
1118,male,female,How much do you care about your partners body ...,"[[0.4493952989578247, 0.5506047010421753]]","[[0.4502927362918854, 0.5497072339057922]]"
1965,american,not american,Do you use counter-clockwise or anti-clockwise?,"[[0.44973525404930115, 0.5502647757530212]]","[[0.45200657844543457, 0.5479934215545654]]"
776,male,female,Do you chew your nails?,"[[0.46014729142189026, 0.5398527383804321]]","[[0.4606046676635742, 0.5393953323364258]]"
154,male,female,Would you rather be a,"[[0.44681796431541443, 0.5531820058822632]]","[[0.44738584756851196, 0.5526140928268433]]"


### WVMCQ7

#### Load list Question with Significant Divisiveness

In [105]:
list_wvs = pd.read_json("https://raw.githubusercontent.com/QuentinTilman/PollBERT-paper/master/Datasets/WVMCQ7/WVMCQ7_significant_divisiveness_questions.json")
list_wvs.columns = ["year","demographic_A","demographic_B","question","p-value"]
list_wvs

,year,demographic_A,demographic_B,question,p-value
0,2018,female,male,Believe in: hell,0.000000e+00
1,2018,female,male,Aims of respondent: first choice,1.000000e-10
2,2018,female,male,"Member: Belong to conservation, the environmen...",2.604500e-06
3,2018,female,male,Member: Belong to sports or recreation,7.723676e-130
4,2018,female,male,"Member: Self-help group, mutual aid group",3.027820e-05
...,...,...,...,...,...
13881,2022,An atheist,Not a religious person,Important in life: Leisure time,3.622743e-04
13882,2022,An atheist,Not a religious person,Believe in: life after death,3.617537e-23
13883,2022,An atheist,Not a religious person,Occupational group - respondent´s spouse (WVS7),9.706773e-04
13884,2022,An atheist,Not a religious person,Democracy: Religious authorities interpret the...,0.000000e+00


In [121]:
sample_wvs = list_wvs.sample(100)

In [157]:
def predict(x):
    demoA = x["demographic_A"]
    demoB = x["demographic_B"]
    year = x["year"]
    question = df_no_demo[df_no_demo["input"] == x["question"]]["question"].values[0]
    options = df_wvs[df_wvs["input"] == x["question"]]["options"].values[0]
    

    qA = demoA+": "+question
    encoding = tokenizer([qA]*len(options), options, return_tensors="pt", padding=True)
    outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()})
    pred_A = outputs.logits.softmax(dim=-1).tolist()

    qB = demoB+": "+question
    encoding = tokenizer([qB]*len(options), options, return_tensors="pt", padding=True)
    outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()})
    pred_B = outputs.logits.softmax(dim=-1).tolist()

    x["pred_A"] = pred_A
    x["pred_B"] = pred_B
    return x

In [158]:
results_sample_wvs = sample_wvs.progress_apply(predict,axis = 1)

  0%|          | 0/100 [00:00<?, ?it/s]

In [162]:
df_wvs[df_wvs["year"] == 2018][df_wvs["input"] == "Future changes: less importance placed on work"][df_wvs["demographic_category"] == "employement"]

C:\Users\Quent\AppData\Local\Temp\ipykernel_34288\1619729487.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_wvs[df_wvs["year"] == 2018][df_wvs["input"] == "Future changes: less importance placed on work"][df_wvs["demographic_category"] == "employement"]


,year,demographic,input,demographic_category,options,labels,split,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5
65636,2018.0,Missing,Future changes: less importance placed on work,employement,"[good, don't mind , bad]","[0.3636363636, 0.1515151515, 0.484848484800000...",train,good,don't mind,bad,,,
66992,2018.0,"No, no paid employment: Retired/pensioned",Future changes: less importance placed on work,employement,"[good, don't mind , bad]","[0.2478498007, 0.2013845186, 0.5507656807, -1....",train,good,don't mind,bad,,,
67147,2018.0,"No, no paid employment:Housewife not otherwise...",Future changes: less importance placed on work,employement,"[good, don't mind , bad]","[0.29409706020000004, 0.14897340180000002, 0.5...",train,good,don't mind,bad,,,
67302,2018.0,"No, no paid employment:Student",Future changes: less importance placed on work,employement,"[good, don't mind , bad]","[0.3130959949, 0.1846789574, 0.5022250477, -1....",train,good,don't mind,bad,,,
67457,2018.0,"No, no paid employment:Unemployed",Future changes: less importance placed on work,employement,"[good, don't mind , bad]","[0.2992156863, 0.1519607843, 0.5488235294, -1....",train,good,don't mind,bad,,,
68590,2018.0,Other,Future changes: less importance placed on work,employement,"[good, don't mind , bad]","[0.2736842105, 0.17543859650000002, 0.55087719...",train,good,don't mind,bad,,,
78258,2018.0,"Yes, has paid employment: Full time employee",Future changes: less importance placed on work,employement,"[good, don't mind , bad]","[0.33596957520000004, 0.1938286011, 0.47020182...",train,good,don't mind,bad,,,
78413,2018.0,"Yes, has paid employment: Part time employee",Future changes: less importance placed on work,employement,"[good, don't mind , bad]","[0.3387243736, 0.1876993166, 0.473576309800000...",train,good,don't mind,bad,,,
78568,2018.0,"Yes, has paid employment: Self employed",Future changes: less importance placed on work,employement,"[good, don't mind , bad]","[0.2949640288, 0.1682443519, 0.5367916193, -1....",train,good,don't mind,bad,,,


In [161]:
results_sample_wvs

,year,demographic_A,demographic_B,question,p-value,pred_A,pred_B
2771,2018,"No, no paid employment:Unemployed","Yes, has paid employment: Self employed",Government has the right: Keep people under vi...,4.004000e-07,"[[0.41169142723083496, 0.0066927107982337475, ...","[[0.41252392530441284, 0.006852209568023682, 0..."
3856,2020,"No, no paid employment: Retired/pensioned","Yes, has paid employment: Part time employee",Highest educational level attained - Responden...,5.294174e-17,"[[0.14735232293605804, 0.15619805455207825, 0....","[[0.14821003377437592, 0.1527046263217926, 0.1..."
8542,2017,"No, no paid employment:Student",Other,Evaluate the impact of immigrants on the devel...,4.000000e-08,"[[0.2876335084438324, 0.281991571187973, 0.156...","[[0.053384747356176376, 0.8311595320701599, 0...."
3976,2020,"No, no paid employment: Retired/pensioned","Yes, has paid employment: Self employed",Highest educational level attained - Responden...,2.206226e-04,"[[0.14735232293605804, 0.15619805455207825, 0....","[[0.14293941855430603, 0.1496550738811493, 0.1..."
681,2018,"No, no paid employment: Retired/pensioned","No, no paid employment:Student",Important child qualities: hard work,1.847768e-19,"[[0.028695352375507355, 0.9713045954704285]]","[[0.5703892707824707, 0.4296106994152069]]"
...,...,...,...,...,...,...,...
3258,2018,"Yes, has paid employment: Part time employee","Yes, has paid employment: Self employed",Future changes: less importance placed on work,2.500000e-07,"[[0.002432079752907157, 0.9951358437538147, 0....","[[0.003065727185457945, 0.9938765168190002, 0...."
6879,2019,"No, no paid employment:Unemployed","Yes, has paid employment: Self employed",Private vs state ownership of business,0.000000e+00,"[[0.4370881915092468, 0.5629117488861084]]","[[0.4520556330680847, 0.5479443669319153]]"
21,2018,female,male,Democracy: People obey their rulers,1.523800e-06,"[[0.3352871537208557, 0.27070263028144836, 0.3...","[[0.33473965525627136, 0.27133283019065857, 0...."
4078,2020,"No, no paid employment:Housewife not otherwise...","No, no paid employment:Student",Private vs state ownership of business,0.000000e+00,"[[0.1592721790075302, 0.8407278060913086]]","[[0.6582905054092407, 0.3417094647884369]]"
